In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyprojroot import here

from knapsack import KnapsackGA
from knapsack.items import items

# Knapsack - Genetic Algorithm

In [ ]:
MAX_WEIGHT = 20
MAX_GENERATIONS = 50

In [ ]:

ga = KnapsackGA(items, max_weight=MAX_WEIGHT, population_size=50)
print(ga)

In [ ]:
solution, result = ga.run(generations=MAX_GENERATIONS)
print(f"{solution=}\n{result.best_fitness=}\n{result.runtime=}s")

These are the items that made the cut:

In [ ]:
print("\n".join([i.name for i in solution]))

## Visualisations

In [ ]:
data = pd.DataFrame([record[:-1] for record in result.history], columns=["generation", "best_fitness", "avg_fitness"])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data, x="generation", y="best_fitness", ax=ax, label="Best value")
sns.lineplot(data, x="generation", y="avg_fitness", ax=ax, label="Average value")

ax.set(title="Results", xlabel="Generation", ylabel="Value")
ax.legend(loc="upper right", bbox_to_anchor=(1.35, 1))

params = ga.params()
param_text = "Parameters\n\n" + "\n".join(f"{k}: {v}" for k, v in params.items())
ax.text(1.05, 0.8, param_text, transform=ax.transAxes, fontsize=10, verticalalignment='top')
fig.tight_layout()

In [ ]:
# Create figures/ directory if it doesn't exist
figures_dir = here("figures")
figures_dir.mkdir(parents=True, exist_ok=True)

# Create timestamped filename
fig_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = figures_dir / f"{fig_timestamp}_ga_results.png"

# Save the figure
fig.savefig(filename, bbox_inches="tight")

### Population 0 distribution

In [ ]:
from knapsack.algorithm import GAResult


def plot_generation_distributions(result: GAResult, generation: int):
    stats = result.history[generation][-1]

    values = [result[0] for result in stats]
    weights = [result[1] for result in stats]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Plot histogram of values
    ax1.hist(values, bins=15, color='lightgreen', edgecolor='black', alpha=0.7)
    ax1.set_title('Value Distribution')
    ax1.set_xlabel('Total Value')
    ax1.set_ylabel('Number of Solutions')

    max_value = max(values)
    ax1.axvline(x=max_value, color='green', linestyle='--', linewidth=2)
    ax1.annotate(f'Best: {max_value:.1f}', 
                    xy=(max_value, ax1.get_ylim()[1]*0.9),
                    xytext=(max_value*0.75, ax1.get_ylim()[1]*0.9),
                    arrowprops=dict(arrowstyle='->'))

    # Plot histogram of weights
    ax2.hist(weights, bins=15, color='skyblue', edgecolor='black', alpha=0.7)
    ax2.set_title('Weight Distribution')
    ax2.set_xlabel('Total Weight')
    ax2.set_ylabel('Number of Solutions')

    ax2.axvline(x=MAX_WEIGHT, color='red', linestyle='--', linewidth=2)
    ax2.annotate(f'Capacity: {MAX_WEIGHT}', 
                    xy=(MAX_WEIGHT, ax2.get_ylim()[1]*0.9),
                    xytext=(MAX_WEIGHT*1.1, ax2.get_ylim()[1]*0.9),
                    arrowprops=dict(arrowstyle='->'))

    fig.suptitle(f"Generation {generation}")
    plt.tight_layout()

    plt.close(fig)
    return fig    

In [ ]:
p0 = plot_generation_distributions(result, 0)
filename = figures_dir / f"{fig_timestamp}_p0_distribution.png"
p0.savefig(filename, bbox_inches="tight")
p0

In [ ]:
pfinal = plot_generation_distributions(result, MAX_GENERATIONS)
filename = figures_dir / f"{fig_timestamp}_pfinal_distribution.png"
pfinal.savefig(filename, bbox_inches="tight")
pfinal